# Reranking - Verbesserte Retrieval-Qualität durch Neuordnung

## Was ist Reranking?

Reranking ist eine **zweistufige Retrieval-Strategie** in RAG-Systemen:

1. **Erste Stufe (Initial Retrieval)**: Schnelle Vorauswahl vieler Dokumente mit einfachen Similarity-Scores (z.B. Cosine-Similarity zwischen Embeddings)
2. **Zweite Stufe (Reranking)**: Präzise Neuordnung dieser Kandidaten mit einem spezialisierten Modell

### Warum ist Reranking wichtig?

**Problem mit reinem Vector Search:**
- Vector Search verwendet **unabhängige Embeddings** für Query und Dokumente
- Die Ähnlichkeit wird durch einen einfachen **Cosine-Score** gemessen
- Dies erfasst oft **nicht die semantische Relevanz** zwischen Query und Dokument

**Beispiel:**
- Query: "Wie funktioniert maschinelles Lernen?"
- Dokument A: "Maschinelles Lernen ist ein wichtiges Thema" (hohe Keyword-Überlappung, aber wenig Inhalt)
- Dokument B: "Algorithmen trainieren Modelle anhand von Daten, um Muster zu erkennen" (erklärt das Konzept, aber weniger Keywords)

Vector Search könnte Dokument A höher ranken, aber Dokument B ist relevanter!

### Lösung: CrossEncoder Reranking

Ein **CrossEncoder** verarbeitet Query und Dokument **gemeinsam** als Paar:
- Eingabe: `[QUERY, DOKUMENT]` als zusammenhängender Text
- Ausgabe: Ein **Relevanz-Score** zwischen -∞ und +∞
  - **Positive Scores** (z.B. +4.0): Dokument ist relevant für die Query
  - **Scores nahe 0** (z.B. -0.5 bis +0.5): Moderate oder unsichere Relevanz
  - **Negative Scores** (z.B. -11.0): Dokument ist irrelevant für die Query
  - Die **absoluten Werte** sind weniger wichtig als die **relative Reihenfolge**
- Vorteil: Kann die **semantische Beziehung** zwischen Query und Dokument besser verstehen

**Trade-offs:**
- ✅ Höhere Qualität der Ergebnisse
- ✅ Bessere semantische Relevanz
- ❌ Langsamer (O(n) statt O(1) wie Vector Search)
- ❌ Kann nur auf kleine Dokumentenmengen angewendet werden

Daher nutzen wir die **zweistufige Strategie**: Erst schnell viele Kandidaten finden, dann präzise reranken!

## Setup: Beispiel-Dokumentensammlung

Wir verwenden die andrena-Dokumentensammlung aus der Hybrid Search Übung. Diese zeigt das **Homonym-Problem**: "andrena" kann sowohl die Softwarefirma **andrena objects** als auch die Wildbienen-Gattung **Andrena** bezeichnen. Ideal, um zu demonstrieren, wie Reranking die Query-Intention besser versteht!

In [13]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os

# Beispiel-Dokumente über andrena objects (Firma) und Andrena (Wildbienen)
# Dieses Homonym-Problem ist ideal, um die Stärken von Reranking zu demonstrieren
documents = [
    # andrena objects - Softwarefirma (5 Dokumente)
    Document(
        page_content="andrena objects ist eine Softwareentwicklungsfirma mit Sitz in Karlsruhe, die sich auf agile Entwicklungsmethoden und Clean Code spezialisiert hat.",
        metadata={"id": "doc1", "topic": "andrena-objects", "category": "company"}
    ),
    Document(
        page_content="andrena objects bietet Dienstleistungen in den Bereichen Software-Entwicklung, Consulting, Coaching und Training für agile Methoden an.",
        metadata={"id": "doc2", "topic": "andrena-objects", "category": "company"}
    ),
    Document(
        page_content="Das Team von andrena objects besteht aus erfahrenen Software-Entwicklern, die Wert auf Qualität und handwerkliches Können legen.",
        metadata={"id": "doc3", "topic": "andrena-objects", "category": "company"}
    ),
    Document(
        page_content="andrena objects setzt auf moderne Technologien wie Java, Python, TypeScript und Cloud-native Architekturen für Kundenprojekte.",
        metadata={"id": "doc4", "topic": "andrena-objects", "category": "company"}
    ),
    Document(
        page_content="Die Unternehmenskultur bei andrena objects fördert kontinuierliches Lernen, Pair Programming und Test-Driven Development.",
        metadata={"id": "doc5", "topic": "andrena-objects", "category": "company"}
    ),

    # Andrena - Wildbienen (6 Dokumente)
    Document(
        page_content="Andrena ist eine Gattung von Wildbienen, die auch als Sandbienen oder Erdbienen bekannt sind und weltweit verbreitet vorkommen.",
        metadata={"id": "doc6", "topic": "andrena-bees", "category": "bees"}
    ),
    Document(
        page_content="Sandbienen der Gattung Andrena nisten in selbstgegrabenen Röhren im Boden und sind wichtige Bestäuber für Obstbäume und Wildpflanzen.",
        metadata={"id": "doc7", "topic": "andrena-bees", "category": "bees"}
    ),
    Document(
        page_content="Die Andrena-Arten sind durch ihre pelzige Behaarung und die Fähigkeit gekennzeichnet, Pollen in speziellen Haarbürsten zu sammeln.",
        metadata={"id": "doc8", "topic": "andrena-bees", "category": "bees"}
    ),
    Document(
        page_content="In Deutschland gibt es über 100 verschiedene Andrena-Arten, die von März bis September aktiv sind und bevorzugt sandige Böden besiedeln.",
        metadata={"id": "doc9", "topic": "andrena-bees", "category": "bees"}
    ),
    Document(
        page_content="Andrena-Weibchen legen Brutzellen an, in denen sie Pollen und Nektar für ihre Nachkommen sammeln, bevor sie ein Ei ablegen.",
        metadata={"id": "doc10", "topic": "andrena-bees", "category": "bees"}
    ),
    Document(
        page_content="Die meisten Andrena-Arten sind oligolektisch, das heißt sie sammeln Pollen nur von bestimmten Pflanzenfamilien wie Weiden oder Kreuzblütlern.",
        metadata={"id": "doc11", "topic": "andrena-bees", "category": "bees"}
    ),

    # Andere Softwarefirmen (5 Dokumente)
    Document(
        page_content="ThoughtWorks ist eine globale Softwareberatung, die sich auf agile Transformation und maßgeschneiderte Software-Lösungen konzentriert.",
        metadata={"id": "doc12", "topic": "other-companies", "category": "company"}
    ),
    Document(
        page_content="Netlight Consulting bietet IT-Beratung und Software-Entwicklung mit Fokus auf digitale Transformation und Innovation in Skandinavien und Europa.",
        metadata={"id": "doc13", "topic": "other-companies", "category": "company"}
    ),
    Document(
        page_content="INNOQ ist eine deutsche IT-Beratungsfirma, die sich auf Architektur-Beratung, Entwicklung und Technologie-Strategie spezialisiert hat.",
        metadata={"id": "doc14", "topic": "other-companies", "category": "company"}
    ),
    Document(
        page_content="it-agile aus Hamburg bietet Schulungen, Coaching und Beratung für Scrum, Kanban und andere agile Frameworks an.",
        metadata={"id": "doc15", "topic": "other-companies", "category": "company"}
    ),
    Document(
        page_content="Zühlke Engineering ist ein internationales Dienstleistungsunternehmen für Innovation und Software-Entwicklung mit Schweizer Wurzeln.",
        metadata={"id": "doc16", "topic": "other-companies", "category": "company"}
    ),

    # Andere Insekten (4 Dokumente)
    Document(
        page_content="Honigbienen der Art Apis mellifera leben in Staaten mit bis zu 50000 Individuen und produzieren Honig als Wintervorrat.",
        metadata={"id": "doc17", "topic": "other-insects", "category": "bees"}
    ),
    Document(
        page_content="Hummeln gehören zur Gattung Bombus und sind robuste Bestäuber, die auch bei kühlen Temperaturen fliegen können.",
        metadata={"id": "doc18", "topic": "other-insects", "category": "bees"}
    ),
    Document(
        page_content="Solitäre Wespen wie die Grabwespen graben Nester im Boden und versorgen ihre Larven mit gelähmten Insekten als Nahrung.",
        metadata={"id": "doc19", "topic": "other-insects", "category": "bees"}
    ),
    Document(
        page_content="Schwebfliegen imitieren das Aussehen von Bienen und Wespen, sind aber harmlose Blütenbesucher und wichtige Bestäuber.",
        metadata={"id": "doc20", "topic": "other-insects", "category": "bees"}
    ),

    # IT-Themen allgemein (5 Dokumente)
    Document(
        page_content="Test-Driven Development ist eine Entwicklungsmethode, bei der Tests vor dem eigentlichen Code geschrieben werden.",
        metadata={"id": "doc21", "topic": "it-general", "category": "tech"}
    ),
    Document(
        page_content="Microservices-Architekturen zerlegen Anwendungen in kleine, unabhängig deploybare Services mit klaren Schnittstellen.",
        metadata={"id": "doc22", "topic": "it-general", "category": "tech"}
    ),
    Document(
        page_content="Domain-Driven Design ist ein Ansatz zur Modellierung komplexer Software-Systeme basierend auf der Fachdomäne.",
        metadata={"id": "doc23", "topic": "it-general", "category": "tech"}
    ),
    Document(
        page_content="Continuous Integration und Continuous Deployment automatisieren den Build- und Deployment-Prozess für schnellere Releases.",
        metadata={"id": "doc24", "topic": "it-general", "category": "tech"}
    ),
    Document(
        page_content="Pair Programming ist eine agile Praxis, bei der zwei Entwickler gemeinsam an einem Computer arbeiten und Code schreiben.",
        metadata={"id": "doc25", "topic": "it-general", "category": "tech"}
    ),
]

print(f"Erstellt {len(documents)} Dokumente")
print("\nBeispiel-Dokument:")
print(f"ID: {documents[0].metadata['id']}")
print(f"Content: {documents[0].page_content}")

Erstellt 25 Dokumente

Beispiel-Dokument:
ID: doc1
Content: andrena objects ist eine Softwareentwicklungsfirma mit Sitz in Karlsruhe, die sich auf agile Entwicklungsmethoden und Clean Code spezialisiert hat.


## Teil 1: Retrieval OHNE Reranking

Zuerst schauen wir uns an, wie ein normaler Vector Search funktioniert und wo seine Limitationen liegen.

In [14]:
import chromadb
from dotenv import load_dotenv

load_dotenv()

# Erstelle Vector Store mit Embeddings
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
# Erstelle einen neuen ephemeral client, um Duplikate bei mehrfacher Ausführung des Notebooks zu vermeiden
client = chromadb.EphemeralClient()
try:
    client.delete_collection("reranking_demo")
except:
    pass

# Erstelle Chroma Vector Store
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="reranking_demo",
    client=client
)

print("✓ Vector Store erstellt")
print(f"  Anzahl Dokumente: {len(documents)}")

✓ Vector Store erstellt
  Anzahl Dokumente: 25


In [15]:
# Query: Wir wollen Informationen über die Softwarefirma andrena objects
query = "wie entwickelt andrena?"

# Retrieval mit Similarity Search (k=5 Dokumente)
results_without_reranking = vector_store.similarity_search_with_score(query, k=7)

print(f"Query: '{query}'")
print("\n" + "="*80)
print("ERGEBNISSE OHNE RERANKING (Vector Similarity)")
print("="*80 + "\n")

for rank, (doc, score) in enumerate(results_without_reranking, 1):
    print(f"Rang {rank} | Similarity: {score:.4f} | ID: {doc.metadata['id']}")
    print(f"Content: {doc.page_content}")
    print("-" * 80)

Query: 'wie entwickelt andrena?'

ERGEBNISSE OHNE RERANKING (Vector Similarity)

Rang 1 | Similarity: 0.7349 | ID: doc1
Content: andrena objects ist eine Softwareentwicklungsfirma mit Sitz in Karlsruhe, die sich auf agile Entwicklungsmethoden und Clean Code spezialisiert hat.
--------------------------------------------------------------------------------
Rang 2 | Similarity: 0.7495 | ID: doc6
Content: Andrena ist eine Gattung von Wildbienen, die auch als Sandbienen oder Erdbienen bekannt sind und weltweit verbreitet vorkommen.
--------------------------------------------------------------------------------
Rang 3 | Similarity: 0.8300 | ID: doc10
Content: Andrena-Weibchen legen Brutzellen an, in denen sie Pollen und Nektar für ihre Nachkommen sammeln, bevor sie ein Ei ablegen.
--------------------------------------------------------------------------------
Rang 4 | Similarity: 0.8421 | ID: doc9
Content: In Deutschland gibt es über 100 verschiedene Andrena-Arten, die von März bis Septem

### Beobachtung: Was ist das Problem?

Schaue dir die Ergebnisse genau an:

**Query:** "wie entwickelt andrena?"

Die Frage zielt eindeutig auf die **Softwarefirma andrena objects** und deren Entwicklungsmethoden ab, nicht auf die biologische Entwicklung von Wildbienen.

**Probleme mit Vector Search:**
1. Dokumente mit **Keyword-Überlappung** ("andrena", "entwickelt") werden hoch gerankt - egal ob Firma oder Bienen
2. Vector Search erfasst nicht die **Query-Intention** (Software-Entwicklung vs. biologische Entwicklung)
3. **Bienen-Dokumente** werden zurückgegeben (siehe Rang 2-5), obwohl sie im Kontext von Software-Entwicklung irrelevant sind

**Das Homonym-Problem:**
- "andrena" alleine ist mehrdeutig (Firma oder Bienen-Gattung)
- "entwickelt" kann sich auf Software-Entwicklung ODER biologische Entwicklung beziehen
- Vector Search kann die richtige Bedeutung nicht aus dem Kontext ableiten

→ **Reranking kann helfen**, weil der CrossEncoder Query und Dokument **gemeinsam** verarbeitet und die Intention besser versteht!

## Teil 2: Retrieval MIT Reranking (CrossEncoder)

Jetzt wenden wir Reranking auf die gleichen Ergebnisse an und schauen, wie sich die Reihenfolge ändert.

**Hinweis:** Falls das Laden des Modells im nächsten Code-Block hängt, führe diesen Befehl im Terminal aus:
```bash
source .venv/bin/activate && python -c "from sentence_transformers import CrossEncoder; CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')"
```

In [16]:
from sentence_transformers import CrossEncoder

# Lade CrossEncoder Modell
# Dieses Modell wurde speziell trainiert, um Query-Document Relevanz zu bewerten
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512)

print("✓ CrossEncoder Modell geladen")

✓ CrossEncoder Modell geladen


In [ ]:
# Reranking: Berechne CrossEncoder Scores für alle Dokumente
print(f"Bereite {len(results_without_reranking)} Dokumente für Reranking vor...")
query_document_pairs = [(query, doc.page_content) for doc, _ in results_without_reranking]

print(f"Berechne CrossEncoder Scores für {len(query_document_pairs)} Query-Document Paare...")
# CrossEncoder gibt einen Relevanz-Score zurück
reranking_scores = cross_encoder.predict(query_document_pairs)
print("✓ CrossEncoder Scores berechnet")

# Kombiniere Dokumente mit neuen Scores
reranked_results = [
    (doc, original_score, rerank_score)
    for (doc, original_score), rerank_score in zip(results_without_reranking, reranking_scores)
]

# Sortiere nach Reranking-Score (höher = besser)
reranked_results = sorted(reranked_results, key=lambda x: x[2], reverse=True)
print("✓ Ergebnisse neu sortiert\n")

print(f"Query: '{query}'")
print("\n" + "="*80)
print("ERGEBNISSE MIT RERANKING (CrossEncoder)")
print("="*80 + "\n")

for rank, (doc, original_score, rerank_score) in enumerate(reranked_results, 1):
    print(f"Rang {rank} | CrossEncoder: {rerank_score:.4f} | Original Similarity: {original_score:.4f} | ID: {doc.metadata['id']}")
    print(f"Content: {doc.page_content}")
    print("-" * 80)

Bereite 7 Dokumente für Reranking vor...
Berechne CrossEncoder Scores für 7 Query-Document Paare...


### Beobachtung: Was hat sich geändert?

Vergleiche die Rankings vorher und nachher:

**Achte auf:**
1. Welche Dokumente haben sich nach oben bewegt?
2. Welche Dokumente haben sich nach unten bewegt?
3. Warum macht das Sinn in Bezug auf die Query?

**Erwartetes Verhalten:**
- **andrena objects Firma-Dokumente** sollten ganz oben sein (höchste Relevanz für die Query)
- **Andrena Bienen-Dokumente** sollten nach unten rutschen oder ganz verschwinden (irrelevant für die Query)
- **Andere Firmen-Dokumente** könnten niedrig gerankt werden (thematisch ähnlich, aber nicht die gesuchte Firma)
- Der CrossEncoder versteht, dass "andrena objects" **als Ganzes** die Firma bezeichnet, nicht nur "andrena"

## Vergleich: Vorher vs. Nachher

Visualisieren wir die Unterschiede zwischen Vector Search und Reranking.

In [18]:
import pandas as pd

# Erstelle Vergleichstabelle
comparison_data = []

for rank, (doc, original_score) in enumerate(results_without_reranking, 1):
    doc_id = doc.metadata['id']
    # Finde das Dokument in den reranked results
    reranked_rank = next(i for i, (d, _, _) in enumerate(reranked_results, 1) if d.metadata['id'] == doc_id)
    rerank_score = next(score for d, _, score in reranked_results if d.metadata['id'] == doc_id)
    
    comparison_data.append({
        'Doc ID': doc_id,
        'Content (Auszug)': doc.page_content[:50] + '...',
        'Rank OHNE Reranking': rank,
        'Vector Similarity': f"{original_score:.4f}",
        'Rank MIT Reranking': reranked_rank,
        'CrossEncoder Score': f"{rerank_score:.4f}",
        'Rang-Änderung': f"{rank - reranked_rank:+d}"
    })

df = pd.DataFrame(comparison_data)
print("\n" + "="*120)
print("VERGLEICH: Vector Search vs. Reranking")
print("="*120)
print(df.to_string(index=False))
print("\n📊 Positive Rang-Änderung = Dokument ist nach oben gerutscht")
print("📊 Negative Rang-Änderung = Dokument ist nach unten gerutscht")

NameError: name 'reranked_results' is not defined

## Übung 2: CONTEXT_SIZE_AFTER_RERANKING Parameter

Nach dem Reranking behalten wir nur die **top K besten Dokumente**, um das Context Window zu reduzieren und Kosten zu sparen.

**Vorteile:**
- Weniger Tokens ans LLM (Kostenersparnis)
- Nur die relevantesten Dokumente
- Entfernung von irrelevantem Noise

Schauen wir uns an, wie sich verschiedene Werte für `CONTEXT_SIZE_AFTER_RERANKING` auswirken.

In [ ]:
def test_context_size_after_reranking(query, initial_k=8, context_size_after=3):
    """
    Zeigt den Effekt von CONTEXT_SIZE_AFTER_RERANKING
    """
    # Initial Retrieval (viele Dokumente)
    results = vector_store.similarity_search_with_score(query, k=initial_k)
    
    # Reranking
    query_document_pairs = [(query, doc.page_content) for doc, _ in results]
    reranking_scores = cross_encoder.predict(query_document_pairs)
    
    reranked = [
        (doc, original_score, rerank_score) 
        for (doc, original_score), rerank_score in zip(results, reranking_scores)
    ]
    reranked = sorted(reranked, key=lambda x: x[2], reverse=True)
    
    # Behalte nur top K nach Reranking
    final_context = reranked[:context_size_after]
    
    print(f"\nQuery: '{query}'")
    print(f"\nPipeline: Initial Retrieval (k={initial_k}) → Reranking → Keep Top {context_size_after}")
    print("\n" + "="*80)
    print("FINALE DOKUMENTE FÜR DAS LLM:")
    print("="*80 + "\n")
    
    for rank, (doc, original_score, rerank_score) in enumerate(final_context, 1):
        print(f"Rang {rank} | CrossEncoder: {rerank_score:.4f} | ID: {doc.metadata['id']}")
        print(f"Content: {doc.page_content}")
        print("-" * 80)
    
    print(f"\n📝 Von {initial_k} initial abgerufenen Dokumenten werden {context_size_after} ans LLM weitergegeben.")
    print(f"💰 Token-Reduktion: ~{((initial_k - context_size_after) / initial_k * 100):.0f}% weniger Context")

# Teste verschiedene Werte
test_context_size_after_reranking(
    query="Wie entwickelt andrena?",
    initial_k=8,
    context_size_after=5
)

In [ ]:
# Experiment: Weniger Dokumente behalten (sehr restriktiv)
test_context_size_after_reranking(
    query="Wie entwickelt andrena?",
    initial_k=10,
    context_size_after=2
)

In [ ]:
# Experiment: Mehr Dokumente behalten
test_context_size_after_reranking(
    query="Wie entwickelt andrena?",
    initial_k=15,
    context_size_after=5
)

### Beobachtung:

**Trade-off bei CONTEXT_SIZE_AFTER_RERANKING:**

**Kleinerer Wert (z.B. 2-3):**
- ✅ Weniger Tokens ans LLM → **Kostenersparnis**
- ✅ Nur die relevantesten Dokumente → **Höhere Precision**
- ❌ Risiko wichtige Informationen zu verlieren → **Niedrigere Recall**

**Größerer Wert (z.B. 5-6):**
- ✅ Mehr Kontext für das LLM → **Höhere Recall**
- ✅ Mehr Sicherheit, relevante Infos einzuschließen
- ❌ Mehr Tokens → **Höhere Kosten**
- ❌ Mehr potentiell irrelevante Dokumente → **Niedrigere Precision**

→ Wähle den Wert basierend auf deinem Use-Case und Budget!

## Reranking im RAG-System aktivieren

Nachdem du Reranking in diesem Notebook getestet hast, möchtest du es jetzt in deinem echten RAG-System aktivieren.

### Schritt 1: `.env` Datei anpassen

Öffne die `.env` Datei im Projekt-Root und setze folgende Variablen:

```bash
# Reranking aktivieren
USE_RERANKING=true

# CrossEncoder Modell (empfohlen für Deutsch/Englisch)
CROSS_ENCODER=cross-encoder/ms-marco-MiniLM-L-12-v2

# Anzahl Dokumente nach Reranking (experimentiere mit 2-6)
CONTEXT_SIZE_AFTER_RERANKING=4

# Stelle sicher, dass initial genug Dokumente abgerufen werden
RETRIEVED_NUMBER_OF_DOCUMENTS=10
```

### Schritt 2: Backend neu starten

Starte das Backend neu, damit die Änderungen wirksam werden:

```bash
uv run --env-file .env python -m src.advanced_rag.backend.main
```

### Schritt 3: Teste im Frontend

Öffne die Web-UI und stelle Fragen. Du solltest sehen:
- Bessere Relevanz der Antworten
- Weniger irrelevante Dokumente im Context

**Tipp:** Vergleiche die gleiche Frage mit `USE_RERANKING=false` und `USE_RERANKING=true`!

## Evaluation: Reranking quantitativ bewerten

Um objektiv zu messen, ob Reranking die Qualität verbessert, führen wir eine systematische Evaluation durch.

### Schritt-für-Schritt Evaluation

#### 1. Baseline: OHNE Reranking evaluieren

Setze in der `.env` Datei:

```bash
USE_RERANKING=false
RETRIEVED_NUMBER_OF_DOCUMENTS=4
```

Starte das Backend:
```bash
uv run --env-file .env python -m src.advanced_rag.backend.main
```

Führe die Evaluation aus (in einem neuen Terminal):
```bash
uv run --env-file .env python src/advanced_rag/evaluation/evaluate_dataset.py
```

**Notiere die Scores aus Langfuse:**
- Context Precision: ?
- Context Recall: ?
- Answer Relevancy: ?
- Faithfulness: ?

#### 2. MIT Reranking evaluieren

Setze in der `.env` Datei:

```bash
USE_RERANKING=true
CROSS_ENCODER=cross-encoder/ms-marco-MiniLM-L-12-v2
RETRIEVED_NUMBER_OF_DOCUMENTS=10  # Initial mehr abrufen
CONTEXT_SIZE_AFTER_RERANKING=4    # Dann auf 4 reduzieren
```

Backend neu starten und Evaluation erneut ausführen.

**Notiere die Scores aus Langfuse:**
- Context Precision: ?
- Context Recall: ?
- Answer Relevancy: ?
- Faithfulness: ?

#### 3. Ergebnisse vergleichen

Trage deine Ergebnisse in diese Tabelle ein:

| Metrik              | OHNE Reranking | MIT Reranking | Verbesserung |
|---------------------|----------------|---------------|---------------|
| Context Precision   |       ?        |       ?       |      ?%       |
| Context Recall      |       ?        |       ?       |      ?%       |
| Answer Relevancy    |       ?        |       ?       |      ?%       |
| Faithfulness        |       ?        |       ?       |      ?%       |

### Erwartete Ergebnisse:

**Context Precision sollte steigen:**
- Reranking entfernt irrelevante Dokumente aus dem Context
- Nur die semantisch relevantesten Dokumente bleiben übrig

**Context Recall könnte leicht sinken:**
- Durch CONTEXT_SIZE_AFTER_RERANKING werden Dokumente entfernt
- Manchmal enthält ein entferntes Dokument relevante Informationen

**Answer Relevancy sollte steigen:**
- Besserer Context führt zu besseren Antworten
- Das LLM wird nicht durch irrelevante Infos abgelenkt

**Faithfulness könnte steigen:**
- Weniger irrelevanter Context reduziert Halluzinationen
- Das LLM bezieht sich auf die wirklich relevanten Dokumente

## Zusammenfassung

### Was hast du gelernt?

1. **Reranking ist eine zweistufige Retrieval-Strategie:**
   - Erste Stufe: Schneller Vector Search mit vielen Kandidaten
   - Zweite Stufe: Präzises Reranking mit CrossEncoder

2. **CrossEncoder vs. Vector Search:**
   - CrossEncoder verarbeitet Query und Dokument gemeinsam
   - Erfasst semantische Relevanz besser als Cosine-Similarity
   - Langsamer, aber qualitativ hochwertiger

3. **CONTEXT_SIZE_AFTER_RERANKING:**
   - Trade-off zwischen Recall (mehr Dokumente) und Precision (bessere Qualität)
   - Experimentiere mit verschiedenen Werten für deinen Use-Case

4. **Evaluation ist wichtig:**
   - Vergleiche Metriken mit und ohne Reranking
   - Context Precision sollte deutlich steigen
   - Answer Relevancy sollte sich verbessern

### Best Practices:

- ✅ Rufe initial mehr Dokumente ab als du final brauchst (z.B. 10 → Rerank → 4)
- ✅ Nutze kleinere CrossEncoder Modelle in Produktion für bessere Latenz
- ✅ Evaluiere systematisch mit echten Test-Queries
- ✅ Monitore die Latenz - Reranking fügt 50-200ms hinzu
- ✅ Experimentiere mit CONTEXT_SIZE_AFTER_RERANKING basierend auf deinem Use-Case

### Weiterführende Fragen:

- Wann lohnt sich Reranking? (Immer bei komplexen Queries!)
- Gibt es Alternativen? (Ja: ColBERT, late interaction models)
- Kann man Reranking mehrfach anwenden? (Ja, aber mit diminishing returns)

Viel Erfolg beim Optimieren deines RAG-Systems! 🚀